<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E5%86%92%E9%9A%AA14_%E6%9B%B4%E5%A4%9A%E7%94%A8%20transformers%20%E7%9A%84%E4%BE%8B%E5%AD%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. 安裝 `transformers` 及準備中文轉換

In [1]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 8.0 MB/s 
     |████████████████████████████████| 895 kB 33.1 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 6.5 MB 27.1 MB/s 
     |████████████████████████████████| 596 kB 51.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install opencc-python-reimplemented

     |████████████████████████████████| 484 kB 7.0 MB/s 
  Created wheel for opencc-python-reimplemented: filename=opencc_python_reimplemented-0.1.6-py2.py3-none-any.whl size=486152 sha256=738fd215cf103bd2e755a97f0fb400eacbdd19799fe3e17747396c09d0c1c5c3
  Stored in directory: /root/.cache/pip/wheels/4e/e2/60/d062d260be08788bb389521544a8fc173de9a9a78d6a593344
Successfully built opencc-python-reimplemented


In [3]:
from transformers import pipeline
from opencc import OpenCC

In [4]:
s2t = OpenCC('s2t').convert
t2s = OpenCC('t2s').convert

In [5]:
t2s('測試一下')

'测试一下'

### 2. 原版的 BERT 來做中文填字

In [6]:
fill_mask = pipeline('fill-mask', model='bert-base-chinese')

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [7]:
fill_mask('生命的真諦是[MASK]。')

[{'score': 0.2658059000968933,
  'sequence': '生 命 的 真 諦 是 愛 。',
  'token': 2695,
  'token_str': '愛'},
 {'score': 0.03982662782073021,
  'sequence': '生 命 的 真 諦 是 你 。',
  'token': 872,
  'token_str': '你'},
 {'score': 0.03092465177178383,
  'sequence': '生 命 的 真 諦 是 美 。',
  'token': 5401,
  'token_str': '美'},
 {'score': 0.03023938275873661,
  'sequence': '生 命 的 真 諦 是 此 。',
  'token': 3634,
  'token_str': '此'},
 {'score': 0.027290023863315582,
  'sequence': '生 命 的 真 諦 是 ： 。',
  'token': 8038,
  'token_str': '：'}]

In [8]:
def fill(text):
    result = fill_mask(text)
    generated_text = []
    for d in result:
        s = ''.join(d['sequence'].split())
        print(s)
        generated_text.append(s)

In [9]:
fill('我天天都會喝一杯[MASK]。')

我天天都會喝一杯酒。
我天天都會喝一杯茶。
我天天都會喝一杯水。
我天天都會喝一杯的。
我天天都會喝一杯奶。


### 3. 閱讀測驗型的問答

這類的問答, 答案一定要出現在文章中才可以。

In [10]:
question_answerer = pipeline("question-answering", model='uer/roberta-base-chinese-extractive-qa')

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/388M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/216 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
context = '''報導隨後介紹本屆奧運金牌得主郭婞淳先前的善舉，除提到郭婞淳名字來自「倖存」一意，也提到郭婞淳在2016年購買救護車送給羅東聖母醫院，後再轉贈離島澎湖惠民醫院的故事。

報導取用澎湖縣長賴峰偉臉書發文的照片，當時郭婞淳奪金後，惠民醫院的醫護人員也覺得與有榮焉。文末則提到郭婞淳是台灣原住民，認為她以阿美族身分一舉奪金，同樣備受關注。'''

In [12]:
question = "郭婞淳捐贈的救護車，最後是在哪個醫院？"

我們寫個函式, 把繁體轉簡體, 送進 model 看看答案是什麼。最後回轉簡體轉繁體的答案。

In [13]:
def QA(question, context):
    c = t2s(context)
    q = t2s(question)
    result = question_answerer(question=q, context=c)
    ans = s2t(result['answer'])
    return ans

In [14]:
QA(question, context)

'離島澎湖惠民醫院'

In [15]:
question = "郭婞淳是哪一個原住民族的人？"

In [16]:
QA(question, context)

'阿美族'

#### 【天馬行空篇】

那我們是不是可以做一個電影訂票系統呢?

In [17]:
context = "你好，我想訂失控的審判兩張票，今天晚上8:20那場，都是全票。"

In [18]:
question = "要看哪場電影？"

In [19]:
QA(question, context)

'失控的審判'

目前相當好! 我們再繼續看下去...

In [20]:
question = "有多少人要看？"

In [21]:
QA(question, context)

'失控'

這也太失控了... 換個問法試試。

In [22]:
question = "要訂幾張票？"

In [23]:
QA(question, context)

'兩張'

In [24]:
question = "看電影的時間？"

In [25]:
QA(question, context)

'你好'

...

### 5. 寫歌機器人

In [26]:
generator = pipeline("text-generation", model="uer/gpt2-chinese-lyric")

Downloading:   0%|          | 0.00/577 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/401M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [29]:
def lyric_generator(text):
    egg=t2s(text)
    lyric = generator(egg, max_length=100)[0]['generated_text']
    return s2t(''.join(lyric.split()))

In [30]:
lyric_generator('回到和妳相遇的地方')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'回到和妳相遇的地方，我的心裏想的全是妳，而妳笑的總是那麼甜，是否我們還有機會能夠再見妳是不是我愛的不夠認真，纔會讓妳離我而去多少個夜裏，我輾轉反側在夢裏，還是回到和妳相遇的地方，我的心裏想的全是妳'

In [31]:
!pip install gradio

     |████████████████████████████████| 649 kB 8.2 MB/s 
     |████████████████████████████████| 1.1 MB 43.9 MB/s 
     |████████████████████████████████| 210 kB 53.4 MB/s 
     |████████████████████████████████| 2.0 MB 44.8 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
     |████████████████████████████████| 84 kB 2.9 MB/s 
     |████████████████████████████████| 255 kB 43.5 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 144 kB 51.6 MB/s 
     |████████████████████████████████| 271 kB 42.6 MB/s 
     |████████████████████████████████| 10.9 MB 35.9 MB/s 
     |████████████████████████████████| 58 kB 6.6 MB/s 
     |████████████████████████████████| 79 kB 8.8 MB/s 
     |████████████████████████████████| 43 kB 2.3 MB/s 
     |████████████████████████████████| 3.6 MB 41.1 MB/s 
     |████████████████████████████████| 856 kB 48.5 MB/s 
     |██████████████████████

In [33]:
import gradio as gr

In [34]:
iface = gr.Interface(lyric_generator,
                     inputs='text',
                     outputs='text')
iface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://14145.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7fac7ca025d0>,
 'http://127.0.0.1:7860/',
 'https://14145.gradio.app')